# ANALYSING NYC HIGH SCHOOL DATA

## BACKGROUND

One of the most controversial issues in the U.S. educational system is the efficacy of the standardized tests, and whether they’re unfair to certain groups. An example of such standardized tests is the SAT, or **Scholastic Aptitude Test**, an exam that U.S. high school students take before applying to college. Colleges take SAT scores into account when deciding who to admit, hence having a high score is important. 

The test comprises three sections, each of which has 800 possible points. The combined score is out of 2,400 possible points (while this number has changed a few times, the dataset for our project is based on 2,400 total points). Organizations often rank high schools by their average SAT scores. The scores are also considered a measure of overall school district quality.


## Data Overview
New York City makes its [data on high school SAT scores](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4) available online, as well as [the demographics for each high school](https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2). The first few rows of the SAT data look like this:
![](https://s3.amazonaws.com/dq-content/sat.png)

With access to this data, we can investigate the correlation between SAT scores and demographics. We could correlate SAT scores with factors like race, gender, income, and more. Unfortunately, combining both of the datasets won't give us all of the demographic information we want to use. We'll need to supplement our data with other sources to conduct a comprehensive analysis.

Luckily, the same website has several related datasets covering demographic information and test scores. Below are the links to all of the datasets we'll be using in our analysis:

1. **[SAT scores by school](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4)** - SAT scores for each high school in New York City.
2. **[School attendance](https://data.cityofnewyork.us/Education/School-Attendance-and-Enrollment-Statistics-by-Dis/7z8d-msnt)** - Attendance information for each school in New York City.
3. **[Class size](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3)** - Information on class size for each school.
4. **[AP test results](https://data.cityofnewyork.us/Education/AP-College-Board-2010-School-Level-Results/itfs-ms3e)** - Advanced Placement (AP) exam results for each high school (passing an optional AP exam in a particular subject can earn a student college credit in that subject).
5. **[Graduation outcomes](https://data.cityofnewyork.us/Education/Graduation-Outcomes-Classes-Of-2005-2010-School-Le/vh2h-md7a)** - The percentage of students who graduated, and other outcome information.
6. **[Demographics](https://data.cityofnewyork.us/Education/School-Demographics-and-Accountability-Snapshot-20/ihfw-zy9j)** - Demographic information for each school.
7. **[School survey](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8)** - Surveys of parents, teachers, and students at each school.

## Analysis Goal
All of these datasets are interrelated. If we can combine them into a single datasetwe may be able to evaluate for some correlations. 

New York City has a significant immigrant population, and is very diverse, so comparing demographic factors such as race, income, and gender with SAT scores is a good way to figure out if the SAT is a fair test. If certain racial groups consistently performed better on the SAT, we would have some evidence that the SAT is unfair, for example.

## LIBRARIES

We will start by importing some useful python libraries. `Numpy` and `Pandas` for performing mathematical operations and manipulating data; `Tabulate` for pretty-printing pandas series and dataframes; the `Plotly` and `matplotlib` visualisation libraries for building informing visuals:

In [1]:
import numpy as np
import pandas as pd
import re
from tabulate import tabulate
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## DATA EXPLORATION AND CLEANING

## Reading the Data

We'll read each file into a pandas dataframe and then store all of the dataframes in a dictionary. This gives us a convenient way to store them and a quick way to reference them later on.

In [2]:
# Create an array of csv file names
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

# Extract corresponding name labels from each csv file
names = [file.split('.')[0] for file in data_files]

# Initialize a new dictionary to hold the datasets.
data = {}

# Loop through data_files to read in the data stored in schools folder
for file, name in zip(data_files, names):
    df= pd.read_csv('schools/{}'.format(file))
    # Assign the filenames as keys and the dataframes as values 
    data[name] = df

## Exploring SAT Data

What we're mainly interested in is the SAT dataset, which corresponds to the dictionary key sat_results. This dataset contains the SAT scores for each high school in New York City. We eventually want to correlate selected information from this dataset with information in the other datasets. 

Let's explore `sat_results` to see what we can discover:

In [3]:
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


### Initial Notes

- *The DBN appears to be a unique ID for each school*.
- *From looking at the first few rows of names, we only have data about high schools*.
- *There's only a single row for each high school, so each DBN is unique in the SAT data*.
- *We may eventually want to combine the three columns that contain SAT scores -- SAT Critical Reading Avg., Score SAT Math Avg. Score, and SAT Writing Avg. Score into a single column to make it easier to analyze*.

Given these observations, let's explore the other datasets to see if we can gain any insight into how to combine them:

In [4]:
for key in data:
    print(data[key].head())

      DBN                             SchoolName AP Test Takers   \
0  01M448           UNIVERSITY NEIGHBORHOOD H.S.              39   
1  01M450                 EAST SIDE COMMUNITY HS              19   
2  01M515                    LOWER EASTSIDE PREP              24   
3  01M539         NEW EXPLORATIONS SCI,TECH,MATH             255   
4  02M296  High School of Hospitality Management               s   

  Total Exams Taken Number of Exams with scores 3 4 or 5  
0                49                                   10  
1                21                                    s  
2                26                                   24  
3               377                                  191  
4                 s                                    s  
   CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE  PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente     0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente     0K          CTT   
2    1

### Additional notes
From the first few rows of each dataset, we can make some observations:

- *Each dataset appears to either have a DBN column or the information we need to create one. That means we can use a DBN column to combine the datasets.*.
- *Some fields look interesting for mapping, particularly Location 1, which contains coordinates inside a larger string*.
- *Some of the datasets appear to contain multiple rows for each school (repeated DBN values), which means we’ll have to do some preprocessing to ensure that DBN is unique within each dataset. If we don't do this, it will cause issues when we combine the datasets, because we might be merging two rows in one dataset with one row in another dataset*.

## Reading the Survey Data

Before we proceed with unifying the datasets, it will be useful to ensure we have all the data we want to unify. We mentioned the survey datasets earlier, but we didn't read them, because they were in a slightly different format with the less common Windows-1252 encoding.

We'll need to specify the encoding and delimiter to the Pandas `read_csv` function in order to ensure the surveys are read in properly. After reading the survey data, we will combine them into a single dataframe using the `pandas.concat()` function:

In [5]:
all_survey = pd.read_csv('schools/survey_all.txt', delimiter='\t', encoding= 'windows-1252')
d75_survey = pd.read_csv('schools/survey_d75.txt', delimiter='\t', encoding= 'windows-1252')
survey = pd.concat([all_survey, d75_survey], axis=0)

survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
0,01M015,M015,P.S. 015 Roberto Clemente,0,No,0.0,Elementary School,NaN,88,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M019,M019,P.S. 019 Asher Levy,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M020,M020,P.S. 020 Anna Silver,0,No,0.0,Elementary School,NaN,88,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M034,M034,P.S. 034 Franklin D. Roosevelt,0,Yes,0.0,Elementary / Middle School,89.0,73,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M063,M063,P.S. 063 William McKinley,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**There are two immediate facts that we can see in our output:**

- There are over 2000 columns, nearly all of which we don't need. We'll have to filter the data to remove the unnecessary ones. Working with fewer columns makes it easier to print the dataframe out and find correlations within it.
- The survey data has a dbn column that we'll want to convert to uppercase (DBN). The conversion makes the column name consistent with the other data sets.

## Cleaning Up the Surveys
First, we’ll need to filter the columns to remove the ones we don’t need. Luckily, there’s a data dictionary at the [original data download location](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8). The dictionary tells us what each column represents. Based on our knowledge of the problem and the analysis we’re trying to do, we can use the data dictionary to determine which columns to use.

Based on the dictionary, it looks like these are the relevant columns:
```
["dbn", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
```

These columns will give us aggregate survey data about how parents, teachers, and students feel about school safety, academic performance, and more. It will also give us the DBN, which allows us to uniquely identify the school.

In [6]:
# Convert the dbn column name to uppercase
survey.rename(columns={'dbn':'DBN'}, inplace=True)

# Extract the relevant survey columns
survey = survey[["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p",
                 "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11",
                 "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11",
                 "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11",
                 "eng_tot_11", "aca_tot_11"]]

# Store the survey data with 'survey' as key in the data dictionary
data['survey'] = survey
survey.head()

,DBN,rr_s,rr_t,rr_p,N_s,N_t,N_p,saf_p_11,com_p_11,eng_p_11,...,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11
0,01M015,NaN,88,60,NaN,22.0,90.0,8.5,7.6,7.5,...,7.6,7.9,NaN,NaN,NaN,NaN,8.0,7.7,7.5,7.9
1,01M019,NaN,100,60,NaN,34.0,161.0,8.4,7.6,7.6,...,8.9,9.1,NaN,NaN,NaN,NaN,8.5,8.1,8.2,8.4
2,01M020,NaN,88,73,NaN,42.0,367.0,8.9,8.3,8.3,...,6.8,7.5,NaN,NaN,NaN,NaN,8.2,7.3,7.5,8.0
3,01M034,89.0,73,50,145.0,29.0,151.0,8.8,8.2,8.0,...,6.8,7.8,6.2,5.9,6.5,7.4,7.3,6.7,7.1,7.9
4,01M063,NaN,100,60,NaN,23.0,90.0,8.7,7.9,8.1,...,7.8,8.1,NaN,NaN,NaN,NaN,8.5,7.6,7.9,8.0


## Inserting DBN Fields
When we explored all of the datasets, we noticed that some of them, like `class_size` and `hs_directory`, don't have a column named `DBN`. However, `hs_directory` has a column named `dbn`, so we can just rename it. `class_size` on the other hand doesn’t appear to have the column at all. Its first three rows are displayed below:

In [7]:
data['class_size'].head(3)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN


From looking at `sat_results` data which does have a `DBN` column, we can tell that the `DBN` is just a combination of the `CSD` and `SCHOOL CODE` columns in the `class_size` data. The main difference is that the `DBN` is padded, so that the `CSD` portion of it always consists of two digits. That means we’ll need to add a leading 0 to the CSD if the CSD is less than two digits long. We can accomplish this using the `pandas.Series.apply()` method, along with a custom function that:

- Takes in a number.
- Converts the number to a string using the `str()` function.
    - Check the length of the string using the `len()` function.
    - If the string is two digits long, returns the string.
    - If the string is one digit long, adds a 0 to the front of the string, then returns it:

In [8]:
data['hs_directory'].rename(columns={'dbn':'DBN'}, inplace=True)


def pad_str(item):
    if len(str(item)) < 2:
        return '0'+str(item)
    else:
        return str(item)
    
data['class_size']['padded_csd'] = data['class_size']['CSD'].apply(pad_str)

data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']

data['class_size'].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01,01M015
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015
